In [22]:
from ga4stpg.util import STEIN_B
from ga4stpg.graph import ReaderORLibrary
from ga4stpg.graph.util import is_steiner_tree

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.mstcrossover import CrossoverPrimUnion
from ga4stpg.tree.generate import GenerateBasedPrimRST
from ga4stpg.tree.mutate import ReplaceByRandomEdge, PrimBasedMutation, Prunning

In [23]:
from ga4stpg.condition import BestKnownReached, Stagnation
from ga4stpg.customevol import GeneticEvolution as Evolution
from ga4stpg.customevol import GeneticPopulation as GPopulation
from ga4stpg.normalization import normalize
from ga4stpg.selector import roullete
from ga4stpg.tracker import DataTracker
from ga4stpg.util import STEIN_B, display, update_best, update_generation

In [24]:
from os import path
import config

In [25]:
# import seaborn as sns
# from matplotlib import pyplot as plt 
# import numpy as np
# import pandas as pd
# from scipy import stats

In [26]:
STEIN_B

[('steinb1.txt', 82),
 ('steinb2.txt', 83),
 ('steinb3.txt', 138),
 ('steinb4.txt', 59),
 ('steinb5.txt', 61),
 ('steinb6.txt', 122),
 ('steinb7.txt', 111),
 ('steinb8.txt', 104),
 ('steinb9.txt', 220),
 ('steinb10.txt', 86),
 ('steinb11.txt', 88),
 ('steinb12.txt', 174),
 ('steinb13.txt', 165),
 ('steinb14.txt', 235),
 ('steinb15.txt', 318),
 ('steinb16.txt', 127),
 ('steinb17.txt', 131),
 ('steinb18.txt', 218)]

In [27]:
INDEX = 7

instance_problem = STEIN_B[INDEX][0]
filename = path.join(config.dataset_directory, instance_problem)
stpg = ReaderORLibrary().parser(filename)

print("STPG information", '\n', 10*'- ','\n')
print('Instance: ', stpg.name)
print('Best Known cost: ', STEIN_B[INDEX][1])
print("Nro. Node:", stpg.nro_nodes)
print("Nro. Edges:", stpg.nro_edges)
print("Nro. Terminals:", stpg.nro_terminals)
print("Terminals: \n", stpg.terminals)

STPG information 
 - - - - - - - - - -  

Instance:  B8
Best Known cost:  104
Nro. Node: 75
Nro. Edges: 94
Nro. Terminals: 19
Terminals: 
 {1, 2, 5, 11, 13, 17, 27, 28, 29, 30, 34, 38, 46, 50, 51, 54, 56, 65, 71}


In [28]:
generator = GenerateBasedPrimRST(stpg)
evaluator = EvaluateTreeGraph(stpg)
crossover = CrossoverPrimUnion(stpg)
prunner   = Prunning(stpg)
mut_prim  = PrimBasedMutation(stpg)
replace_random = ReplaceByRandomEdge(stpg)

In [29]:
output_data_dir = path.join("data", stpg.name)
tracker = DataTracker('001',target=output_data_dir)

In [30]:
population = (GPopulation(
        chromosomes=[ generator() for _ in range(50)],
        eval_function=evaluator,
        maximize=True)
    .evaluate()
    .normalize(norm_function=normalize)
    .callback(update_best))

In [31]:
evol = (Evolution()
        .evaluate()
        .normalize(norm_function=normalize)
        .callback(update_best)
        .callback(tracker.log_evaluation)
        .select(selection_func=roullete)
        .crossover(combiner=crossover)
        .mutate(mutate_function=replace_random, probability=0.2)
        .mutate(mutate_function=mut_prim, probability=0.2)
        .mutate(mutate_function=prunner, probability=0.2)
        .callback(update_generation)
        .callback(display, every=100))

In [32]:
%%time
result = population.evolve(evol, n=2_000)

Population 256976 | size 50 | generation 100 | best cost 104
Population 256976 | size 50 | generation 200 | best cost 104
Population 256976 | size 50 | generation 300 | best cost 104
Population 256976 | size 50 | generation 400 | best cost 104
Population 256976 | size 50 | generation 500 | best cost 104
Population 256976 | size 50 | generation 600 | best cost 104
Population 256976 | size 50 | generation 700 | best cost 104
Population 256976 | size 50 | generation 800 | best cost 104
Population 256976 | size 50 | generation 900 | best cost 104
Population 256976 | size 50 | generation 1000 | best cost 104
Population 256976 | size 50 | generation 1100 | best cost 104
Population 256976 | size 50 | generation 1200 | best cost 104
Population 256976 | size 50 | generation 1300 | best cost 104
Population 256976 | size 50 | generation 1400 | best cost 104
Population 256976 | size 50 | generation 1500 | best cost 104
Population 256976 | size 50 | generation 1600 | best cost 104
Population 256976

In [33]:
tracker.report()

In [34]:
best_overall = result.documented_best

In [35]:
is_steiner_tree(best_overall.chromosome, stpg)

(True,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': True})

In [36]:
best_overall.cost

104

In [37]:
best_overall.fitness

243

In [38]:
pop = result.evaluate().normalize(norm_function=normalize)

In [39]:
pop.current_best

In [40]:
current_best = result.current_best

In [41]:
current_best.fitness

5

In [42]:
current_best.cost

107